In [6]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.linalg import norm
from scipy.integrate import odeint
import pprint
from ipywidgets import interact, interactive, fixed, interact_manual, jslink
import ipywidgets as widgets
import ipyvolume as ipv
import ipyvolume.pylab as p3
import traitlets
import time, sys
from IPython.core.display import clear_output

In [278]:
# define the atoms in the chain (this test will only use a 3 atom chain)
r1 = (0, 0, 1)
r2 = (2, 1, 1)
r3 = (3, 2, 1)
r4 = (4, 4, 1)

floaters = np.hstack((r1, r2, r3, r4))

m = 4             # total atoms in the chain

In [279]:
# define parameters
beta = 1

In [294]:
# define time interval
t = np.linspace(0,6,20)

In [299]:
# bending force DE system
def bending_force(floaters, t):
    
    current_bending_force = np.zeros((m,3))
    total_bending_force = np.zeros(m)
    
    # loop for computing bending force
    for i in range(1,m-1):          # loop through each atom not on the end of the chain and compute the bending force on it and its neighbors
        
        # define the pieces of the bending energy equation
        sx_n = floaters[i*3] - floaters[(i-1)*3]
        sx_n1 = floaters[(i+1)*3] - floaters[i*3]

        sy_n = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        sy_n1 = floaters[(i+1)*3 + 1] - floaters[i*3 + 1]

        sz_n = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        sz_n1 = floaters[(i+1)*3 + 2] - floaters[i*3 + 2]

        A = norm((sx_n, sy_n, sz_n))*norm((sx_n1, sy_n1, sz_n1))

        B = np.dot((sx_n, sy_n, sz_n),(sx_n1, sy_n1, sz_n1))

        partial_common = .5/A
        
        # partials of A wrt x for atom n, n-1 and n+1, respectively
        Axn = partial_common*(2*(sx_n)*(sx_n1**2) - 2*(sx_n1)*(sx_n**2) + 2*(sx_n)*(sy_n1**2) + 2*(sx_n)*(sz_n1**2) - 2*(sx_n1)*(sy_n**2) - 2*(sx_n1)*(sz_n**2))
        Axn0 = partial_common*(-2*(sx_n)*(sx_n1**2) - 2*(sx_n)*(sy_n1**2) - 2*(sx_n)*(sz_n1**2))
        Axn1 = partial_common*(2*(sx_n**2)*(sx_n1) + 2*(sy_n**2)*(sx_n1) + 2*(sz_n**2)*(sx_n1))
        
        # partials of A wrt y for atom n, n-1 and n+1, respectively
        Ayn = partial_common*(-2*(sx_n**2)*(sy_n1) + 2*(sy_n)*(sx_n1**2) + 2*(sy_n)*(sy_n1**2) - 2*(sy_n**2)*(sy_n1) + 2*(sy_n)*(sz_n1**2) - 2*(sz_n**2)*(sy_n1))
        Ayn0 = partial_common*(-2*(sy_n)*(sx_n1**2) - 2*(sy_n)*(sy_n1**2) - 2*(sy_n)*(sz_n1**2))
        Ayn1 = partial_common*(2*(sy_n**2)*(sy_n1) + 2*(sz_n**2)*(sy_n1) + 2*(sx_n**2)*(sy_n1))
        
        # partials of A wrt z for atom n, n-1 and n+1, respectively
        Azn = partial_common*(-2*(sx_n**2)*(sz_n1) - 2*(sy_n**2)*(sz_n1) - 2*(sz_n**2)*(sz_n1) + 2*(sz_n)*(sz_n1**2) + 2*(sz_n)*(sx_n1**2) + 2*(sz_n)*(sy_n1**2))
        Azn0 = partial_common*(-2*(sz_n)*(sx_n1**2) -2*(sz_n)*(sy_n1**2) - 2*(sz_n)*(sz_n1**2))
        Azn1 = partial_common*(2*(sz_n1)*(sx_n**2) + 2*(sz_n1)*(sy_n**2) + 2*(sz_n1)*(sz_n**2))
        
        # partials of B wrt x for atom n, n-1 and n+1, respectively
        Bxn = floaters[(i+1)*3] - 2*floaters[i*3] + floaters[(i-1)*3]
        Bxn0 = -floaters[(i+1)*3] + floaters[i*3]
        Bxn1 = floaters[i*3] - floaters[(i-1)*3]

        # partials of B wrt y for atom n, n-1 and n+1, respectively
        Byn = floaters[(i+1)*3 + 1] - 2*floaters[i*3 + 1] + floaters[(i-1)*3 + 1]
        Byn0 = -floaters[(i+1)*3 + 1] + floaters[i*3 + 1]
        Byn1 = floaters[i*3 + 1] - floaters[(i-1)*3 + 1]
        
        # partials of B wrt z for atom n, n-1 and n+1, respectively
        Bzn = floaters[(i+1)*3 + 2] - 2*floaters[i*3 + 2] + floaters[(i-1)*3 + 2]
        Bzn0 = -floaters[(i+1)*3 + 2] + floaters[i*3 + 2]
        Bzn1 = floaters[i*3 + 2] - floaters[(i-1)*3 + 2]
        
        try:
            Eb_common = 4*beta/((A+B)**2)                   # compute common part of bending energy gradient
            if (np.isnan(Eb_common)  or np.isinf(Eb_common) or np.isnan(A)):
                raise ValueError('Divide by zero')
                raise Exception
            
        except Exception as error:
            print("Caught exception: ", error)
            continue
            
        current_bending_force[i][0] += Eb_common*(B*Axn - A*Bxn)
        current_bending_force[i][1] += Eb_common*(B*Ayn - A*Byn)
        current_bending_force[i][2] += Eb_common*(B*Azn - A*Bzn)
            
        current_bending_force[i-1][0] += Eb_common*(B*Axn0 - A*Bxn0)
        current_bending_force[i-1][1] += Eb_common*(B*Ayn0 - A*Byn0)
        current_bending_force[i-1][2] += Eb_common*(B*Azn0 - A*Bzn0)
            
        current_bending_force[i+1][0] = Eb_common*(B*Axn1 - A*Bxn1)
        current_bending_force[i+1][1] = Eb_common*(B*Ayn1 - A*Byn1)
        current_bending_force[i+1][2] = Eb_common*(B*Azn1 - A*Bzn1)

    total_bending_force = np.hstack(current_bending_force)
    print(total_bending_force)
    return total_bending_force

In [300]:
%%time

# solve the system of ODEs to compute changing positions as a result of the bending force acting on the chain
gFlow = odeint(bending_force, floaters, t, atol=1.4e-10)

[-0.06662043  0.13324087  0.          0.06662043 -0.13324087  0.
  0.13324087 -0.06662043  0.         -0.13324087  0.06662043  0.        ]
[-0.06662053  0.13324144  0.          0.06662053 -0.13324144  0.
  0.13324144 -0.06662053  0.         -0.13324144  0.06662053  0.        ]
[-0.06662053  0.13324144  0.          0.06662053 -0.13324144  0.
  0.13324144 -0.06662053  0.         -0.13324144  0.06662053  0.        ]
[-0.06662063  0.13324201  0.          0.06662063 -0.13324201  0.
  0.13324201 -0.06662063  0.         -0.13324201  0.06662063  0.        ]
[-0.06662063  0.13324201  0.          0.06662063 -0.13324201  0.
  0.13324201 -0.06662063  0.         -0.13324201  0.06662063  0.        ]
[-0.06664967  0.13340997  0.          0.06664967 -0.13340997  0.
  0.13340997 -0.06664967  0.         -0.13340997  0.06664967  0.        ]
[-0.06664967  0.13340997  0.          0.06664967 -0.13340997  0.
  0.13340997 -0.06664967  0.         -0.13340997  0.06664967  0.        ]
[-0.06667864  0.13357817  0

[ 3.49988841  1.63984966  0.         -3.49988841 -1.63984967  0.
  1.63984966  3.4998884  -0.         -1.63984966 -3.4998884  -0.        ]
[ 3.78496677  1.65305377  0.         -3.78496677 -1.65305377  0.
  1.65305376  3.78496677 -0.         -1.65305377 -3.78496677 -0.        ]
[ 3.78496829  1.65305395  0.         -3.78496829 -1.65305395  0.
  1.65305394  3.78496829 -0.         -1.65305394 -3.78496828 -0.        ]
[ 4.11896162  1.66182082  0.         -4.11896162 -1.66182082  0.
  1.66182081  4.11896162 -0.         -1.66182081 -4.11896161 -0.        ]
[ 4.11896497  1.66182113  0.         -4.11896497 -1.66182113  0.
  1.66182113  4.11896497 -0.         -1.66182113 -4.11896496 -0.        ]
[ 4.05004918  1.66057188  0.         -4.05004917 -1.66057188  0.
  1.66057187  4.05004917 -0.         -1.66057188 -4.05004917 -0.        ]
[ 4.05005071  1.66057203  0.         -4.05005071 -1.66057203  0.
  1.66057203  4.05005071 -0.         -1.66057203 -4.0500507  -0.        ]
[ 4.35426375  1.66400871  0

[ 1529.94999161 -1114.62369217     0.         -1529.94988333  1114.62358389
     0.         -1114.62355481  1529.94979963    -0.          1114.62344653
 -1529.94969135    -0.        ]
[ 1529.95555721 -1114.62820457     0.         -1529.95544893  1114.62809629
     0.         -1114.62806721  1529.95536523    -0.          1114.62795892
 -1529.95525695    -0.        ]
[ 1504.41847864 -1093.92671388     0.         -1504.41837378  1093.92660903
     0.         -1093.92658109  1504.41829264    -0.          1093.92647623
 -1504.41818778    -0.        ]
[ 1504.42116497 -1093.92888913     0.         -1504.42106011  1093.92878427
     0.         -1093.92875634  1504.42097896    -0.          1093.92865148
 -1504.4208741     -0.        ]
[ 1627.92255004 -1194.27613703     0.         -1627.92242814  1194.27601514
     0.         -1194.27598143  1627.92233428    -0.          1194.27585953
 -1627.92221238    -0.        ]
[ 1627.92299143 -1194.27649654     0.         -1627.92286953  1194.27637464
    

[  9.52707798e+08  -9.49389233e+08   0.00000000e+00  -9.44994658e+08
   9.41676077e+08   0.00000000e+00  -9.37215047e+08   9.40490983e+08
  -0.00000000e+00   9.29501908e+08  -9.32777828e+08  -0.00000000e+00]
[  9.14883459e+08  -9.11653307e+08   0.00000000e+00  -9.07726166e+08
   9.04496001e+08   0.00000000e+00  -9.00356723e+08   9.03546784e+08
  -0.00000000e+00   8.93199431e+08  -8.96389477e+08  -0.00000000e+00]
[  9.14962486e+08  -9.11732149e+08   0.00000000e+00  -9.07803323e+08
   9.04572972e+08   0.00000000e+00  -9.00432612e+08   9.03622849e+08
  -0.00000000e+00   8.93273449e+08  -8.96463671e+08  -0.00000000e+00]
[  1.05047503e+09  -1.04693319e+09   0.00000000e+00  -1.04124209e+09
   1.03770024e+09   0.00000000e+00  -1.03235939e+09   1.03585182e+09
  -0.00000000e+00   1.02312646e+09  -1.02661886e+09  -0.00000000e+00]
[  1.05057607e+09  -1.04703401e+09   0.00000000e+00  -1.04134041e+09
   1.03779833e+09   0.00000000e+00  -1.03245590e+09   1.03594854e+09
  -0.00000000e+00   1.02322023

  -0.00000000e+00   1.49415103e+11  -1.49519597e+11  -0.00000000e+00]
[  9.21175312e+15  -9.21208386e+15   0.00000000e+00  -5.64296903e+15
   5.64311318e+15   0.00000000e+00  -3.56893407e+15   3.56912077e+15
  -0.00000000e+00   1.49984995e+11  -1.50089787e+11  -0.00000000e+00]
[  9.53723045e+15  -9.53757477e+15   0.00000000e+00  -5.84234872e+15
   5.84249983e+15   0.00000000e+00  -3.69503175e+15   3.69522508e+15
  -0.00000000e+00   1.50025848e+11  -1.50130661e+11  -0.00000000e+00]
[  9.62912614e+15  -9.62947431e+15   0.00000000e+00  -5.89864185e+15
   5.89879492e+15   0.00000000e+00  -3.73063433e+15   3.73082953e+15
  -0.00000000e+00   1.50037051e+11  -1.50141870e+11  -0.00000000e+00]
[  7.74403559e+15  -7.74430565e+15   0.00000000e+00  -4.74387996e+15
   4.74399333e+15   0.00000000e+00  -3.00030540e+15   3.00046220e+15
  -0.00000000e+00   1.49773780e+11  -1.49878461e+11  -0.00000000e+00]
[  7.81649017e+15  -7.81676319e+15   0.00000000e+00  -4.78826393e+15
   4.78837879e+15   0.0000000

...

[  2.67560898e+23  -2.67575085e+23   0.00000000e+00  -1.63900906e+23
   1.63909553e+23   0.00000000e+00  -1.03660001e+23   1.03665532e+23
  -0.00000000e+00   1.53589477e+11  -1.53696148e+11  -0.00000000e+00]
[ -3.42075503e+22   3.42093968e+22   0.00000000e+00   2.09546546e+22
  -2.09558255e+22   0.00000000e+00   1.32529002e+22  -1.32535938e+22
  -0.00000000e+00   1.53591498e+11  -1.53698170e+11  -0.00000000e+00]
Caught exception:  Divide by zero
[  0.00000000e+00   0.00000000e+00   0.00000000e+00  -5.95231232e+10
   5.95262910e+10   0.00000000e+00  -9.40650944e+10   9.41685970e+10
   0.00000000e+00   1.53588218e+11  -1.53694888e+11  -0.00000000e+00]
[  8.41574152e+21  -8.41624592e+21   0.00000000e+00  -5.15525248e+21
   5.15559475e+21   0.00000000e+00  -3.26049354e+21   3.26065117e+21
  -0.00000000e+00   1.53591212e+11  -1.53697884e+11  -0.00000000e+00]
[ -3.42075503e+22   3.42093968e+22   0.00000000e+00   2.09546546e+22
  -2.09558255e+22   0.00000000e+00   1.32529002e+22  -1.32535938e

  -3.04763897e+21   1.53608877e+11  -1.53715558e+11   1.75047100e+06]
[ -1.31537542e+23   1.31544556e+23   0.00000000e+00   8.05764950e+22
  -8.05807959e+22   0.00000000e+00   5.09610485e+22  -5.09637610e+22
  -0.00000000e+00   1.53591074e+11  -1.53697749e+11  -0.00000000e+00]
[ -3.81430775e+22   3.81451110e+22   0.00000000e+00   2.33654619e+22
  -2.33667092e+22   0.00000000e+00   1.47776156e+22  -1.47784018e+22
  -0.00000000e+00   1.53624617e+11  -1.53731304e+11  -0.00000000e+00]
[ -5.36239955e+22   5.36268545e+22   4.97513494e+21   3.28486711e+22
  -3.28504251e+22  -3.04763897e+21   2.07753242e+22  -2.07764296e+22
  -1.92749597e+21   1.53608877e+11  -1.53715558e+11   2.76773830e+06]
[ -5.36239955e+22   5.36268545e+22   0.00000000e+00   3.28486711e+22
  -3.28504251e+22   0.00000000e+00   2.07753242e+22  -2.07764296e+22
  -0.00000000e+00   1.53627230e+11  -1.53733920e+11  -0.00000000e+00]
[ -5.36239955e+22   5.36268545e+22   0.00000000e+00   3.28486711e+22
  -3.28504251e+22   0.0000000

[  2.98076017e+22  -2.98091847e+22   0.00000000e+00  -1.82593665e+22
   1.82603348e+22   0.00000000e+00  -1.15482353e+22   1.15488498e+22
  -0.00000000e+00   1.53567105e+11  -1.53673764e+11  -0.00000000e+00]
[  3.46524753e+22  -3.46543156e+22   0.00000000e+00  -2.12272106e+22
   2.12283364e+22   0.00000000e+00  -1.34252646e+22   1.34259789e+22
  -0.00000000e+00   1.53568053e+11  -1.53674713e+11  -0.00000000e+00]
[  4.24956327e+22  -4.24978900e+22   0.00000000e+00  -2.60317261e+22
   2.60331075e+22   0.00000000e+00  -1.64639066e+22   1.64647827e+22
  -0.00000000e+00   1.53568206e+11  -1.53674866e+11  -0.00000000e+00]
[  3.95323655e+22  -3.95344651e+22   0.00000000e+00  -2.42165055e+22
   2.42177896e+22   0.00000000e+00  -1.53158602e+22   1.53166753e+22
  -0.00000000e+00   1.53569815e+11  -1.53676476e+11  -0.00000000e+00]
[  5.02539217e+22  -5.02565916e+22   0.00000000e+00  -3.07842534e+22
   3.07858873e+22   0.00000000e+00  -1.94696685e+22   1.94707044e+22
  -0.00000000e+00   1.53569720

[ -1.07932907e+23   1.07938661e+23  -3.93271386e+22   6.61169059e+22
  -6.61204333e+22   2.40907878e+22   4.18160025e+22  -4.18182270e+22
   1.52363508e+22   1.53602842e+11  -1.53709520e+11  -0.00000000e+00]
[ -4.21637984e+23   4.21660443e+23   0.00000000e+00   2.58284524e+23
  -2.58298282e+23   0.00000000e+00   1.63353463e+23  -1.63362159e+23
  -0.00000000e+00   1.53602291e+11  -1.53708969e+11  -0.00000000e+00]
[ -1.11433801e+23   1.11439740e+23   0.00000000e+00   6.82614626e+22
  -6.82651049e+22   0.00000000e+00   4.31723389e+22  -4.31746360e+22
  -0.00000000e+00   1.53603604e+11  -1.53710282e+11  -0.00000000e+00]
[ -1.07932907e+23   1.07938661e+23   2.40907878e+22   6.61169059e+22
  -6.61204333e+22  -1.47573934e+22   4.18160025e+22  -4.18182270e+22
  -9.33339436e+21   1.53602842e+11  -1.53709520e+11   1.75037930e+06]
[ -3.00987849e+23   3.01003893e+23   0.00000000e+00   1.84377369e+23
  -1.84387205e+23   0.00000000e+00   1.16610482e+23  -1.16616688e+23
  -0.00000000e+00   1.53585041

/anaconda/lib/python3.6/site-packages/scipy/integrate/odepack.py:218: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)


In [301]:
print(gFlow)

[[  0.00000000e+000   0.00000000e+000   1.00000000e+000   2.00000000e+000
    1.00000000e+000   1.00000000e+000   3.00000000e+000   2.00000000e+000
    1.00000000e+000   4.00000000e+000   4.00000000e+000   1.00000000e+000]
 [ -2.15673445e-002   4.56871562e-002   1.00000000e+000   2.02156734e+000
    9.54312844e-001   1.00000000e+000   3.04568716e+000   1.97843266e+000
    1.00000000e+000   3.95431284e+000   4.02156734e+000   1.00000000e+000]
 [ -4.38840981e-002   9.96176964e-002   1.00000000e+000   2.04388410e+000
    9.00382304e-001   1.00000000e+000   3.09961770e+000   1.95611590e+000
    1.00000000e+000   3.90038230e+000   4.04388410e+000   1.00000000e+000]
 [ -6.62404688e-002   1.63650092e-001   1.00000000e+000   2.06624047e+000
    8.36349908e-001   1.00000000e+000   3.16365009e+000   1.93375953e+000
    1.00000000e+000   3.83634991e+000   4.06624047e+000   1.00000000e+000]
 [ -8.74352562e-002   2.40222093e-001   1.00000000e+000   2.08743526e+000
    7.59777907e-001   1.00000000e+

In [302]:
sx_n = gFlow[-1][3] - gFlow[-1][0]
sx_n1 = gFlow[-1][6] - gFlow[-1][3]

sy_n = gFlow[-1][4] - gFlow[-1][1]
sy_n1 = gFlow[-1][7] - gFlow[-1][4]

sz_n = gFlow[-1][5] - gFlow[-1][2]
sz_n1 = gFlow[-1][8] - gFlow[-1][5]

A = norm((sx_n, sy_n, sz_n))*norm((sx_n1, sy_n1, sz_n1))

B = np.dot((sx_n, sy_n, sz_n),(sx_n1, sy_n1, sz_n1))

print(A, B, A-B, A+B, 2*beta*(A-B)/(A+B))

inf -inf inf nan nan


/Users/christopher/.anaconda/ipykernel/ipykernel_launcher.py:14: RuntimeWarning: invalid value encountered in double_scalars
  
